### Disease-[share_variants]-Disease
- Task: Map UMLS Disease to UMLS Disease based on variants shared between diseaes (Jaccard)
- Source: DisGeNet, disease_to_disease_CURATED.tsv


Not sure if this should be used

In [8]:
import pandas as pd
import csv
import json
import requests as req
import xml.etree.ElementTree as ET
import biomedkg_utils
import time
import datetime
from biomedkg_utils import *
import os

In [10]:
# Download or redownload files
try: 
    os.system('rm input/disease_to_disease_CURATED.tsv.gz')
except: 
    pass
try:
    os.system('rm input/disease_to_disease_CURATED.tsv')
except:
    pass
os.system('wget -N -P input/ https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz')
os.system('gunzip input/disease_to_disease_CURATED.tsv.gz')

rm: cannot remove 'input/disease_to_disease_CURATED.tsv.gz': No such file or directory
rm: cannot remove 'input/disease_to_disease_CURATED.tsv': No such file or directory
--2023-06-13 05:54:39--  https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz
Resolving www.disgenet.org (www.disgenet.org)... 84.89.134.141
Connecting to www.disgenet.org (www.disgenet.org)|84.89.134.141|:443... gzip: input/disease_to_disease_CURATED.tsv.gz: No such file or directory


256

In [ ]:
os.system('wget -N -P input/ https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz')
os.system('gunzip input/disease_to_disease_CURATED.tsv.gz')

--2023-06-13 05:54:46--  https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz
Resolving www.disgenet.org (www.disgenet.org)... 84.89.134.141
Connecting to www.disgenet.org (www.disgenet.org)|84.89.134.141|:443... failed: Connection timed out.
Retrying.

--2023-06-13 05:56:57--  (try: 2)  https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz
Connecting to www.disgenet.org (www.disgenet.org)|84.89.134.141|:443... failed: Connection timed out.
Retrying.

--2023-06-13 05:59:09--  (try: 3)  https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz
Connecting to www.disgenet.org (www.disgenet.org)|84.89.134.141|:443... failed: Connection timed out.
Retrying.

--2023-06-13 06:01:23--  (try: 4)  https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_to_disease_CURATED.tsv.gz
Connecting to www.disgenet.org (www.disgenet.org)|84.89.134.141|:443... failed: Connecti

In [ ]:
dda_df = pd.read_table('input/disease_to_disease_CURATED.tsv')[['diseaseId1','diseaseId2','jaccard_genes','jaccard_variant']]
print(len(dda_df), 'rows in shared genes and variants dataframe')

In [ ]:
'''Shared genes'''
dda_gene_df = dda_df.query('jaccard_genes != 0')

# UMLS Disease IDs
with open('output/disease2disease/edges_umlsDisease-SHARES_GENES-umlsDisease.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Disease (UMLS)','Disease (UMLS)','Relationship','Weight'])
    
    for disease1, disease2, jac_gene in zip(dda_gene_df['diseaseId1'], dda_gene_df['diseaseId2'], dda_gene_df['jaccard_genes']):
        if jac_gene > 0:
            writer.writerow(['UMLS_Disease:'+disease1, 'UMLS_Disease:'+disease2, '-diseases_share_genes-', jac_gene])
            
dda_gene_df = pd.read_csv('output/disease2disease/edges_umlsDisease-SHARES_geneS-umlsDisease.csv').drop_duplicates()
dda_gene_df.to_csv('output/disease2disease/edges_umlsDisease-SHARES_GENES-umlsDisease.csv', index=False)
! cp 'output/disease2disease/edges_umlsDisease-SHARES_GENES-umlsDisease.csv' 'output/edges/edges_umlsDisease-SHARES_GENES-umlsDisease.csv'
print(len(dda_gene_df), 'disease pairs with shared genes')
print(len(set(dda_gene_df['Disease (UMLS)']).union(set(dda_gene_df['Disease (UMLS).1']))), 'diseases that share genes with another disease')

In [ ]:
'''Shared variants'''
dda_var_df = dda_df.query('jaccard_variant != 0')

# UMLS Disease IDs
with open('output/disease2disease/edges_umlsDisease-SHARES_VARIANTS-umlsDisease.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Disease (UMLS)','Disease (UMLS)','Relationship','Weight'])
    
    for disease1, disease2, jac_variant in zip(dda_var_df['diseaseId1'], dda_var_df['diseaseId2'], dda_var_df['jaccard_variant']):
        if jac_variant > 0:
            writer.writerow(['UMLS_Disease:'+disease1, 
                             'UMLS_Disease:'+disease2, 
                             '-diseases_share_variants-', 
                             jac_variant])
        else:
            print(jac_variant)
            
dda_var_df = pd.read_csv('output/disease2disease/edges_umlsDisease-SHARES_VARIANTS-umlsDisease.csv').drop_duplicates()
dda_var_df.to_csv('output/disease2disease/edges_umlsDisease-SHARES_VARIANTS-umlsDisease.csv', index=False)
! cp 'output/disease2disease/edges_umlsDisease-SHARES_VARIANTS-umlsDisease.csv' 'output/edges/edges_umlsDisease-SHARES_VARIANTS-umlsDisease.csv'
print(len(dda_var_df), 'disease pairs with shared variants')
print(len(set(dda_var_df['Disease (UMLS)']).union(set(dda_var_df['Disease (UMLS).1']))), 'diseases that share variants with another disease')

In [ ]:
umls2mesh = json.load(open('output/otherMappings/umls2mesh.json'))

In [6]:
# MeSH Disease IDs
file = 'Disease_(MeSH)_sharesvariants_Disease_(MeSH).csv'
outpath = os.path.join('output/disease2disease/',file)
with open(outpath, 'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Disease (MeSH)',
                     'Disease (MeSH)',
                     'Relationship',
                     'Weight'])
    
    with open('input/disease_to_disease_CURATED.tsv') as fin:
        for i,line in enumerate(fin):
            if i == 0:
                continue
            line = line.strip().split('\t')
            try:
                dis1s = umls2mesh[line[0]]
                dis2s = umls2mesh[line[1]]
                jacvar = float(line[12])
                if jacvar <= 0:
                    continue
            except:
                continue
            for dis1 in dis1s:
                for dis2 in dis2s:
                    writer.writerow(['MeSH_Disease:'+dis1, 
                                     'MeSH_Disease:'+dis2, 
                                     '-diseases_share_variants-', 
                                     jacvar])
                    
df = pd.read_csv(outpath)
df.to_csv(os.path.join('output/edges', file), index=False)
df.to_csv(os.path.join('output/edges to use', file), index=False)

In [7]:
# MeSH Disease IDs
file = 'Disease_(MeSH)_sharesgenes_Disease_(MeSH).csv'
outpath = os.path.join('output/disease2disease/',file)
with open(outpath, 'w', newline='') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Disease (MeSH)','Disease (MeSH)','Relationship','Weight'])
    with open('input/disease_to_disease_CURATED.tsv') as fin:
        for i,line in enumerate(fin):
            if i == 0:
                continue
            line = line.strip().split('\t')
            try:
                dis1s = umls2mesh[line[0]]
                dis2s = umls2mesh[line[1]]
                jac_gen = float(line[12])
                if jac_gen <= 0:
                    continue
            except:
                continue
            for dis1 in dis1s:
                for dis2 in dis2s:
                    writer.writerow(['MeSH_Disease:'+dis1, 'MeSH_Disease:'+dis2, '-diseases_share_genes-', jac_gen])

df = pd.read_csv(outpath)
df.to_csv(os.path.join('output/edges', file), index=False)
df.to_csv(os.path.join('output/edges to use', file), index=False)

### Disease: MeSH -[is]- MeSH
- Name -[is]- ID
- ID -[is]- Tree Number
- Tree Number -[is] - Name

In [3]:
year = datetime.datetime.now().year

tree = ET.parse(f'input/desc{year}.xml')
root = tree.getroot()

In [8]:
year = datetime.datetime.now().year
os.system(f'wget -N -P input/ https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/desc{year}.xml')
tree = ET.parse(f'input/desc{year}.xml')
root = tree.getroot()

--2023-03-23 16:30:54--  https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/desc2023.xml
Resolving nlmpubs.nlm.nih.gov (nlmpubs.nlm.nih.gov)... 130.14.173.134
Connecting to nlmpubs.nlm.nih.gov (nlmpubs.nlm.nih.gov)|130.14.173.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306680626 (292M) [text/xml]
Saving to: ‘input/desc2023.xml’

     0K .......... .......... .......... .......... ..........  0%  347K 14m22s
    50K .......... .......... .......... .......... ..........  0%  345K 14m25s
   100K .......... .......... .......... .......... ..........  0%  696K 12m0s
   150K .......... .......... .......... .......... ..........  0%  693K 10m48s
   200K .......... .......... .......... .......... ..........  0%  695K 10m4s
   250K .......... .......... .......... .......... ..........  0% 52.2M 8m24s
   300K .......... .......... .......... .......... ..........  0% 89.6M 7m13s
   350K .......... .......... .......... .......... ..........  0%  1

  7050K .......... .......... .......... .......... ..........  2%  985K 42s
  7100K .......... .......... .......... .......... ..........  2% 21.7M 41s
  7150K .......... .......... .......... .......... ..........  2%  123M 41s
  7200K .......... .......... .......... .......... ..........  2%  159M 41s
  7250K .......... .......... .......... .......... ..........  2% 72.7M 41s
  7300K .......... .......... .......... .......... ..........  2%  191M 40s
  7350K .......... .......... .......... .......... ..........  2%  180M 40s
  7400K .......... .......... .......... .......... ..........  2%  107M 40s
  7450K .......... .......... .......... .......... ..........  2% 78.9M 39s
  7500K .......... .......... .......... .......... ..........  2%  187M 39s
  7550K .......... .......... .......... .......... ..........  2%  196M 39s
  7600K .......... .......... .......... .......... ..........  2%  164M 39s
  7650K .......... .......... .......... .......... ..........  2%  194M 38s

 16400K .......... .......... .......... .......... ..........  5% 1.21M 25s
 16450K .......... .......... .......... .......... ..........  5% 53.9M 25s
 16500K .......... .......... .......... .......... ..........  5% 67.3M 25s
 16550K .......... .......... .......... .......... ..........  5% 26.3M 25s
 16600K .......... .......... .......... .......... ..........  5% 85.1M 25s
 16650K .......... .......... .......... .......... ..........  5%  173M 24s
 16700K .......... .......... .......... .......... ..........  5%  173M 24s
 16750K .......... .......... .......... .......... ..........  5%  166M 24s
 16800K .......... .......... .......... .......... ..........  5%  145M 24s
 16850K .......... .......... .......... .......... ..........  5%  170M 24s
 16900K .......... .......... .......... .......... ..........  5% 45.5M 24s
 16950K .......... .......... .......... .......... ..........  5%  172M 24s
 17000K .......... .......... .......... .......... ..........  5%  102M 24s

 25900K .......... .......... .......... .......... ..........  8%  847K 20s
 25950K .......... .......... .......... .......... ..........  8%  190M 20s
 26000K .......... .......... .......... .......... ..........  8%  148M 20s
 26050K .......... .......... .......... .......... ..........  8% 91.0M 20s
 26100K .......... .......... .......... .......... ..........  8%  114M 20s
 26150K .......... .......... .......... .......... ..........  8% 48.1M 20s
 26200K .......... .......... .......... .......... ..........  8% 63.6M 20s
 26250K .......... .......... .......... .......... ..........  8% 55.3M 19s
 26300K .......... .......... .......... .......... ..........  8%  104M 19s
 26350K .......... .......... .......... .......... ..........  8% 64.4M 19s
 26400K .......... .......... .......... .......... ..........  8%  164M 19s
 26450K .......... .......... .......... .......... ..........  8% 73.2M 19s
 26500K .......... .......... .......... .......... ..........  8% 64.3M 19s

 35500K .......... .......... .......... .......... .......... 11%  855K 17s
 35550K .......... .......... .......... .......... .......... 11%  197M 17s
 35600K .......... .......... .......... .......... .......... 11% 33.0M 17s
 35650K .......... .......... .......... .......... .......... 11%  113M 17s
 35700K .......... .......... .......... .......... .......... 11% 79.0M 17s
 35750K .......... .......... .......... .......... .......... 11% 99.9M 17s
 35800K .......... .......... .......... .......... .......... 11%  167M 17s
 35850K .......... .......... .......... .......... .......... 11% 40.6M 17s
 35900K .......... .......... .......... .......... .......... 12% 71.0M 17s
 35950K .......... .......... .......... .......... .......... 12% 47.5M 17s
 36000K .......... .......... .......... .......... .......... 12% 69.9M 17s
 36050K .......... .......... .......... .......... .......... 12%  112M 17s
 36100K .......... .......... .......... .......... .......... 12%  134M 17s

...

264650K .......... .......... .......... .......... .......... 88%  874K 2s
264700K .......... .......... .......... .......... .......... 88% 85.2M 2s
264750K .......... .......... .......... .......... .......... 88%  199M 2s
264800K .......... .......... .......... .......... .......... 88% 77.2M 2s
264850K .......... .......... .......... .......... .......... 88% 88.0M 2s
264900K .......... .......... .......... .......... .......... 88%  117M 2s
264950K .......... .......... .......... .......... .......... 88%  205M 2s
265000K .......... .......... .......... .......... .......... 88%  123M 2s
265050K .......... .......... .......... .......... .......... 88% 99.8M 2s
265100K .......... .......... .......... .......... .......... 88%  101M 2s
265150K .......... .......... .......... .......... .......... 88%  207M 2s
265200K .......... .......... .......... .......... .......... 88% 87.9M 2s
265250K .......... .......... .......... .......... .......... 88%  126M 2s
265300K ....

274150K .......... .......... .......... .......... .......... 91%  868K 1s
274200K .......... .......... .......... .......... .......... 91% 56.3M 1s
274250K .......... .......... .......... .......... .......... 91%  107M 1s
274300K .......... .......... .......... .......... .......... 91%  211M 1s
274350K .......... .......... .......... .......... .......... 91% 53.3M 1s
274400K .......... .......... .......... .......... .......... 91% 76.1M 1s
274450K .......... .......... .......... .......... .......... 91%  140M 1s
274500K .......... .......... .......... .......... .......... 91%  126M 1s
274550K .......... .......... .......... .......... .......... 91% 84.3M 1s
274600K .......... .......... .......... .......... .......... 91% 64.3M 1s
274650K .......... .......... .......... .......... .......... 91%  111M 1s
274700K .......... .......... .......... .......... .......... 91%  207M 1s
274750K .......... .......... .......... .......... .......... 91%  144M 1s
274800K ....

283700K .......... .......... .......... .......... .......... 94% 1.03M 1s
283750K .......... .......... .......... .......... .......... 94%  198M 1s
283800K .......... .......... .......... .......... .......... 94%  112M 1s
283850K .......... .......... .......... .......... .......... 94% 80.5M 1s
283900K .......... .......... .......... .......... .......... 94% 87.3M 1s
283950K .......... .......... .......... .......... .......... 94%  203M 1s
284000K .......... .......... .......... .......... .......... 94% 76.4M 1s
284050K .......... .......... .......... .......... .......... 94% 62.1M 1s
284100K .......... .......... .......... .......... .......... 94% 71.8M 1s
284150K .......... .......... .......... .......... .......... 94%  125M 1s
284200K .......... .......... .......... .......... .......... 94%  177M 1s
284250K .......... .......... .......... .......... .......... 94%  178M 1s
284300K .......... .......... .......... .......... .......... 94% 74.8M 1s
284350K ....

293200K .......... .......... .......... .......... .......... 97%  932K 0s
293250K .......... .......... .......... .......... .......... 97% 74.0M 0s
293300K .......... .......... .......... .......... .......... 97% 83.6M 0s
293350K .......... .......... .......... .......... .......... 97%  132M 0s
293400K .......... .......... .......... .......... .......... 97%  102M 0s
293450K .......... .......... .......... .......... .......... 97% 64.8M 0s
293500K .......... .......... .......... .......... .......... 98%  103M 0s
293550K .......... .......... .......... .......... .......... 98%  206M 0s
293600K .......... .......... .......... .......... .......... 98% 79.2M 0s
293650K .......... .......... .......... .......... .......... 98% 99.3M 0s
293700K .......... .......... .......... .......... .......... 98%  124M 0s
293750K .......... .......... .......... .......... .......... 98%  145M 0s
293800K .......... .......... .......... .......... .......... 98%  119M 0s
293850K ....

2023-03-23 16:31:09 (20.6 MB/s) - ‘input/desc2023.xml’ saved [306680626/306680626]



In [9]:
name2id, id2name, id2tree, tree2id = dict(), dict(), dict(), dict()
all_tree_numbers = list()

for ele in root:
    try:
        # MeSH Tree Number
        tree_numbers = ele.find('TreeNumberList').findall('TreeNumber')
        
        # If anatomy
        for tree_number in tree_numbers:
            if tree_number.text.startswith(('C','F03')): # exclude animal diseases C22?
                
                '''
                Tree
                '''
                tree_number = tree_number.text
                all_tree_numbers.append(tree_number)
                
                
                '''
                ID to Tree
                '''
                try:
                    # MeSH ID
                    ID = ele.find('DescriptorUI').text

                    # MeSH ID -[is]- MeSH Tree
                    id2tree.setdefault(ID,set()).add(tree_number)
                    tree2id.setdefault(tree_number,set()).add(ID)
                except:
                    pass
                    
                '''
                ID to Name
                '''
                try:
                    # MeSH ID
                    ID = ele.find('DescriptorUI').text

                    # MeSH Term Name
                    name = ele.find('DescriptorName').find('String').text

                    # MeSH Term -[is]- MeSH ID
                    name2id.setdefault(name,set()).add(ID)
                    id2name.setdefault(ID,set()).add(name)
                except:
                    pass
    except:
        continue        
        
all_tree_numbers = sorted(all_tree_numbers)
tree2id = dict(sorted(tree2id.items()))

for k,v in name2id.copy().items():
    name2id[k] = list(name2id[k])

In [10]:
'''MeSH Tree is MeSH Term/Name'''
tree2id = switch_dictset_to_dictlist(tree2id)
tree2name = dict()

for tree, ID in tree2id.items():
    assert len(ID) == 1

    name_list = list(id2name[ID[0]])
    assert len(name_list) == 1
    name = name_list[0]

    tree2name.setdefault(tree,list()).append(name)
    
for tree, name in tree2name.items():
    assert len(name) == 1
    
for tree, name in tree2name.copy().items():
    tree2name[tree] = name[0]
    
#json.dump(tree2name, open('output/disease2disease/tree2name.json','w'))

In [11]:
# MeSH Tree Number -[is]- MeSH ID
file = 'Disease_(MeSH)_2_Disease_(MeSH_Tree).csv'
output_edgefile_onerel_noweight(outpath = os.path.join('output/disease2disease',file),
                                columns = ['Disease (MeSH)','Disease (MeSH Tree)','Relationship'],
                                dictionary = id2tree,
                                rel = '-is-',
                                prefix_col1 = 'MeSH_Disease:',
                                prefix_col2 = 'MeSH_Tree_Disease:')

df = pd.read_csv(os.path.join('output/disease2disease',file)).drop_duplicates()
df.to_csv(os.path.join('output/edges to use/', file), index=False)

In [12]:
# MeSH Term -[is]- MeSH ID
with open('output/disease2disease/meshterm-IS-meshid.json','w') as fout:
    json.dump(name2id, fout)

In [13]:
mesh_name2id = json.load(open('output/disease2disease/meshterm-IS-meshid.json'))

### Disease: MeSH Tree Number -[child_of]-> MeSH Tree Number

In [14]:
tree2tree = dict()

# Tree Number
for tree_num in all_tree_numbers:
    if '.' in tree_num:
        
        # Parent of Tree Number
        parent = ''
        for num in tree_num.split('.')[:len(tree_num.split('.'))-1]:
            parent += num+'.'
        parent = parent.strip('.')
        
        # Tree Number -[subclass of]-> Tree Number
        tree2tree[tree_num] = [parent]

In [16]:
# MeSH Tree Number -[subclass of]-> MeSH Tree Number
output_edgefile_onerel_noweight(outpath = 'output/disease2disease/edges_meshtree-SUBCLASS->meshtree.csv',
                                columns = ['Disease (MeSH Tree)','Disease (MeSH Tree)','Relationship'],
                                dictionary = tree2tree,
                                rel = '-subclass_of->',
                                prefix_col1 = 'MeSH_Tree_Disease:',
                                prefix_col2 = 'MeSH_Tree_Disease:')

df = pd.read_csv('output/disease2disease/edges_meshtree-SUBCLASS->meshtree.csv')
df.tail()

,Disease (MeSH Tree),Disease (MeSH Tree).1,Relationship
13151,MeSH_Tree_Disease:F03.950.750.375.500,MeSH_Tree_Disease:F03.950.750.375,-subclass_of->
13152,MeSH_Tree_Disease:F03.950.750.438,MeSH_Tree_Disease:F03.950.750,-subclass_of->
13153,MeSH_Tree_Disease:F03.950.750.438.500,MeSH_Tree_Disease:F03.950.750.438,-subclass_of->
13154,MeSH_Tree_Disease:F03.950.750.500,MeSH_Tree_Disease:F03.950.750,-subclass_of->
13155,MeSH_Tree_Disease:F03.950.750.550,MeSH_Tree_Disease:F03.950.750,-subclass_of->


In [17]:
len(mondo2mesh)

5563

### Disease: DOID-is-MeSH
- also in compound_to_disease

In [ ]:
! wget -N -P input/ https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo

doid2mesh = dict()
with open('input/HumanDO.obo') as fin:
    IDs = []
    meshes = []
    for i,line in enumerate(fin):
        if i < 29:
            continue
        
        # New DOID term
        if '[Term]' in line:
            if len(IDs) > 0:
                meshes = set(meshes)
                for ID in IDs:
                    for mesh in meshes:
                        doid2mesh.setdefault(ID,set()).add(mesh)
                IDs = []
                meshes = []
                
        # DOID IDs
        elif 'id' in line.split(':')[0]:
            IDs.append(line.split(':')[1].strip()+':'+line.split(':')[2].strip())
        
        # Get UMLS CUIs
        if 'xref: UMLS_CUI:' in line:
            umls = line.split(':')[2].strip()
            try: meshes += umls2mesh[umls]
            except: continue
        
        # Get MeSH via UMLS CUIs
        if 'xref: MESH:' in line:
            mesh = line.split(':')[2].strip()
            meshes.append(mesh)

            
for k in doid2mesh.copy():
    doid2mesh[k] = list(doid2mesh[k])
        
with open('output/disease2disease/doid2mesh.json','w') as fout:
    json.dump(doid2mesh, fout)

# OMIM is MeSH
OMIM -> MONDO + MONDO -> MeSH = OMIM -> MeSH

In [ ]:
!wget -N -P input/ 'http://purl.obolibrary.org/obo/mondo/mondo-with-equivalents.json'

In [ ]:
mondo2others = json.load(open('input/mondo-with-equivalents.json'))

In [ ]:
omim2mesh, omim2umls, omim2doid, omim2mondo = dict(), dict(), dict(), dict()
mondo2mesh, mondo2umls, mondo2doid, mondo2omim = dict(), dict(), dict(), dict()
mesh2mondo = dict()
mondos = set()

entries = mondo2others['graphs'][0]['equivalentNodesSets']
for i in range(len(entries)):
    node = mondo2others['graphs'][0]['equivalentNodesSets'][i]
    
    umls, doid, mesh, omim = '','','',''
    for id_link in node['nodeIds']:
        id_link = id_link.split('/')
        if '/umls/' in id_link:
            umls = id_link[-1]
        elif 'DOID' in id_link[-1]:
            doid = id_link[-1]
        elif 'MESH_' in id_link[-1]:
            mesh = id_link[-1]
        elif 'MONDO_' in id_link[-1]:
            mondo = id_link[-1]
        elif 'OMIM_' in id_link[-1]:
            omim = id_link[-1]
            
    if omim != '':
        if mesh != '':
            omim2mesh.setdefault(omim, set()).add(mesh)
        if umls != '':
            omim2umls.setdefault(omim, set()).add(umls)
        if doid != '':
            omim2doid.setdefault(omim, set()).add(doid)
        if mondo != '':
            omim2mondo.setdefault(omim, set()).add(mondo)
            
    if mondo != '':
        if mesh != '':
            mondo2mesh.setdefault(mondo, set()).add(mesh)
            mesh2mondo.setdefault(mesh, set()).add(mondo)
        if umls != '':
            mondo2umls.setdefault(mondo, set()).add(umls)
        if doid != '':
            mondo2doid.setdefault(mondo, set()).add(doid)
        if omim != '':
            mondo2omim.setdefault(mondo, set()).add(omim)
        mondos.add(mondo)
mondo_list = mondos

In [ ]:
#omim2mesh = dict()
mesh2omim = dict()
for omim, mondos in omim2mondo.items():
    
    # MONDO
    for mondo in mondos:
        if mondo in mondo2mesh:
            
            # MeSH
            meshes = mondo2mesh[mondo]
            for mesh in meshes:
                mesh = mesh.split('MESH_')[1]
                
                # OMIM - MeSH
                omim2mesh.setdefault(omim, set()).add(mesh)
                mesh2omim.setdefault(mesh, set()).add(omim)
                

                
print(len(omim2mesh), len(mesh2omim))
json.dump(switch_dictset_to_dictlist(omim2mesh), open('output/disease2disease/omim2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mondo2omim), open('output/disease2disease/mondo2omim.json','w'))

mondos = {mondo.replace('_',':') for mondo in mondo_list}
mondo_list = list(mondos)
mesh2mondo = {mesh.replace('MESH_',''):{mondo.replace('_',':') for mondo in mondos} for mesh,mondos in mesh2mondo.items()}
mondo2mesh = {mondo.replace('_',':'):{mesh.replace('MESH_','') for mesh in meshes} for mondo,meshes in mondo2mesh.items()}

In [ ]:
''' OMIM - UMLS - MeSH '''
omim2mesh = json.load(open('output/disease2disease/omim2mesh.json'))
omim2mesh = switch_dictlist_to_dictset(omim2mesh)

# UMLS - OMIM
umls2omim = dict()
with open('input/MRCONSO.RRF') as fin:
    for line in fin:
        line = line.strip().split('|')
        umls = line[0]
        ontology = line[11]
        
        if 'OMIM' in ontology:
            disease = (line[12] == 'PT')
            if disease:
                omim_id = line[13]
                umls2omim.setdefault(umls, set()).add(omim_id)
                
                
# OMIM - UMLS - MeSH
print(len(omim2mesh), 'OMIM-MeSH Before')
print(len(mesh2omim), 'MeSH-OMIM Before')
umls2mesh = json.load(open('output/otherMappings/umls2mesh.json'))

# UMLS
for umls, omims in umls2omim.items():
    if umls in umls2mesh:
        
        # MeSH
        meshes = umls2mesh[umls]
        
        # OMIM
        for omim in omims:
            
            # OMIM - MeSH
            mesh2omim.setdefault(mesh, set()).add(omim)
            omim2mesh.setdefault(omim, set()).add(mesh)
            
print(len(omim2mesh), 'OMIM-MeSH After')
print(len(mesh2omim), 'MeSH-OMIM After')

In [ ]:
def get_meshes(temp_mesh, meshes):
    if type(temp_mesh) == list:
        temp_meshes = temp_mesh
    else:
        temp_meshes = [temp_mesh]
    
    for mesh in temp_meshes:
        meshes.append(mesh)
    
    return meshes



BATCH_SIZE = 500
for batch_i in range(0,int(len(mondo_list)/BATCH_SIZE)):
    start_ind = batch_i*500
    end_ind = (batch_i+1)*500
    if end_ind > len(mondo_list):
        end_ind = len(mondo_list)-1
    
    print(start_ind, end_ind)
    
    params = {'q': ','.join(mondo_list[start_ind:end_ind])}
    res = requests.post('http://mydisease.info/v1/query', params).json()
    time.sleep(3)
    
    for i in range(0,len(res)):

        ''' MONDO '''
        mondo = res[i]['query']


        ''' OMIM '''
        try:
            omim = res[i]['disease_ontology']['xrefs']['omim']
        except:
            try:
                omim = res[i]['mondo']['xrefs']['omim']
            except:
                try:
                    omim = res[i]['disgenet']['xrefs']['omim'] 
                except:
                    try:
                        omim = res[i]['hpo']['omim']
                    except:
                        continue
                        
                        
        ''' OMIM - MONDO '''
        if type(omim) == list:
            omims = omim
        else:
            omims = [omim]

        for omim in omims:
            omim2mondo.setdefault(omim, set()).add(mondo)
            mondo2omim.setdefault(mondo, set()).add(omim)


        ''' MeSH '''
        meshes = list()
        try: meshes = get_meshes(res[i]['umls']['mesh']['preferred'], meshes)
        except: pass
        
        try: meshes = get_meshes(res[i]['mondo']['xrefs']['mesh'], meshes)
        except: pass
        
        try: meshes = get_meshes(res[i]['ctd']['mesh'], meshes)
        except: pass
        
        try: meshes = get_meshes(res[i]['disease_ontology']['xrefs']['mesh'], meshes)
        except: pass
            
        try: meshes = get_meshes(res[i]['disgenet']['xrefs']['mesh'], meshes)
        except: pass
        
        try: meshes = get_meshes(res[i]['umls']['mesh']['non-preferred'], meshes)
        except: pass

        ''' MeSH - MONDO '''
        for mesh in meshes:
            mesh2mondo.setdefault(mesh, set()).add(mondo)
            mondo2mesh.setdefault(mondo, set()).add(mesh)

In [ ]:
'''OMIM MeSH'''
# MONDO
for mondo, meshes in mondo2mesh.items():
    if mondo in mondo2omim:
        
        # OMIM
        omims = mondo2omim[mondo]
        
        # MeSH
        for mesh in meshes:
            
            # OMIM - MeSH
            omim2mesh.setdefault(omim, set()).add(mesh)
            mesh2omim.setdefault(mesh, set()).add(omim)
            
print(len(omim2mesh), len(mesh2omim))

In [ ]:
print(len(mondo2mesh), len(mesh2mondo))

In [ ]:
json.dump(switch_dictset_to_dictlist(omim2mesh), open('output/disease2disease/omim2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2omim), open('output/disease2disease/mesh2omim.json','w'))
json.dump(switch_dictset_to_dictlist(mondo2mesh), open('output/disease2disease/mondo2mesh.json','w'))
json.dump(switch_dictset_to_dictlist(mesh2mondo), open('output/disease2disease/mesh2mondo.json','w'))